In [3]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import re
import pickle
from itertools import combinations
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [26]:
df_results = pd.read_csv('all_results_df.csv')

In [17]:
df = pd.read_csv("cleaned_car_price_prediction_door_fix.csv")

In [18]:
df['HasTurbo'] = df['HasTurbo'].astype(int)
df.columns = [re.sub(' ', '_', col) for col in df.columns]
df.columns = [re.sub('\.', '', col) for col in df.columns]
x_cols = df.columns[1:]
df = df.sample(n=2000, random_state=42) # select a subset
df.head()
df.Doors.value_counts()

<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:3: SyntaxWarning: invalid escape sequence '\.'
/var/folders/d4/3_k3cjj111n8n_ybx72scyb40000gn/T/ipykernel_85631/3876706845.py:3: SyntaxWarning: invalid escape sequence '\.'
  df.columns = [re.sub('\.', '', col) for col in df.columns]


Doors
4     1913
2       81
>5       6
Name: count, dtype: int64

In [19]:
df.head()

,Price,Manufacturer,Prod_year,Category,Leather_interior,Fuel_type,Engine_Volume,HasTurbo,Mileage,Cylinders,Gear_box_type,Drive_wheels,Doors,Wheel,Color,Airbags
736,27284,CHEVROLET,2014,Hatchback,No,Plug-in Hybrid,1.4,0,65000,4.0,Automatic,Front,4,Left wheel,Silver,10
8674,10349,MERCEDES-BENZ,1997,Microbus,Yes,Diesel,2.9,1,3333,6.0,Manual,Rear,2,Left wheel,Red,2
5971,40769,MERCEDES-BENZ,1996,Sedan,No,Petrol,1.8,0,212485,8.0,Manual,Rear,4,Left wheel,Green,2
1957,38737,HYUNDAI,2014,Jeep,Yes,Diesel,2.0,0,132756,4.0,Automatic,Front,4,Left wheel,Grey,4
11075,42102,SSANGYONG,2017,Jeep,Yes,Petrol,1.6,0,50750,4.0,Automatic,Front,4,Left wheel,White,4


0   Price             2000 non-null   int64  
 1   Manufacturer      2000 non-null   object 
 2   Prod_year         2000 non-null   int64  
 3   Category          2000 non-null   object 
 4   Leather_interior  2000 non-null   object 
 5   Fuel_type         2000 non-null   object 
 6   Engine_Volume     2000 non-null   float64
 7   HasTurbo          2000 non-null   int64  
 8   Mileage           2000 non-null   int64  
 9   Cylinders         2000 non-null   float64
 10  Gear_box_type     2000 non-null   object 
 11  Drive_wheels      2000 non-null   object 
 12  Doors             2000 non-null   object 
 13  Wheel             2000 non-null   object 
 14  Color             2000 non-null   object 
 15  Airbags           2000 non-null   int64 

In [23]:
cv_model_regress_string = "Price ~ C(Manufacturer) + Prod_year + C(Category) + C(Fuel_type) + C(HasTurbo) + Cylinders + C(Gear_box_type) + C(Drive_wheels) + C(Wheel) + Airbags"

In [41]:
cv_model = smf.ols(cv_model_regress_string, data = df).fit()

In [42]:
cv_model.params

Intercept                      -2.999863e+06
C(Manufacturer)[T.AUDI]         3.045043e+03
C(Manufacturer)[T.BMW]          1.224378e+04
C(Manufacturer)[T.BUICK]       -2.878595e+02
C(Manufacturer)[T.CADILLAC]     7.504806e+03
                                    ...     
C(Drive_wheels)[T.Rear]         2.470202e+03
C(Wheel)[T.Right-hand drive]   -1.087654e+03
Prod_year                       1.497024e+03
Cylinders                       2.033034e+03
Airbags                        -8.523103e+02
Length: 70, dtype: float64

In [43]:
aic_model_preds = df_results.iloc[28221]

In [39]:
aic_model_preds['Predictors']

"['Manufacturer', 'Prod_year', 'Category', 'Fuel_type', 'Engine_Volume', 'HasTurbo', 'Cylinders', 'Gear_box_type', 'Drive_wheels', 'Airbags']"

In [44]:
aic_model_string = "Price ~ C(Manufacturer) + Prod_year + C(Category) + C(Fuel_type) + Engine_Volume + C(HasTurbo) + Cylinders + C(Gear_box_type) + C(Drive_wheels) + Airbags"

In [45]:
aic_model = smf.ols(aic_model_string, data = df).fit()

In [49]:
aic_model.params

Intercept                     -3.034394e+06
C(Manufacturer)[T.AUDI]        3.756034e+03
C(Manufacturer)[T.BMW]         1.310446e+04
C(Manufacturer)[T.BUICK]       1.145679e+03
C(Manufacturer)[T.CADILLAC]    7.885624e+03
                                   ...     
C(Drive_wheels)[T.Rear]        2.747306e+03
Prod_year                      1.512949e+03
Engine_Volume                  1.700510e+03
Cylinders                      1.296979e+03
Airbags                       -8.778789e+02
Length: 70, dtype: float64

In [50]:
bic_model_preds = df_results.loc[7694]

In [52]:
bic_model_preds['Predictors']

"['Prod_year', 'Fuel_type', 'Engine_Volume', 'HasTurbo', 'Gear_box_type', 'Airbags']"

In [55]:
bic_model_string = "Price ~ Prod_year + C(Fuel_type) + Engine_Volume + C(HasTurbo) +  C(Gear_box_type) + Airbags"

In [56]:
bic_model = smf.ols(bic_model_string, data = df).fit()

In [57]:
bic_model.params

Intercept                        -2.725383e+06
C(Fuel_type)[T.Diesel]            2.358172e+03
C(Fuel_type)[T.Hybrid]           -8.030168e+03
C(Fuel_type)[T.LPG]              -7.495083e+03
C(Fuel_type)[T.Petrol]           -4.127678e+03
C(Fuel_type)[T.Plug-in Hybrid]    2.597030e+03
C(HasTurbo)[T.1]                  7.469564e+03
C(Gear_box_type)[T.Manual]        5.468235e+03
C(Gear_box_type)[T.Tiptronic]     1.138413e+04
C(Gear_box_type)[T.Variator]      6.809069e+03
Prod_year                         1.363534e+03
Engine_Volume                     2.881329e+03
Airbags                          -8.528417e+02
dtype: float64

In [58]:
r2a_model_preds = df_results.iloc[28221]

In [60]:
r2a_model_preds["Predictors"]

"['Manufacturer', 'Prod_year', 'Category', 'Fuel_type', 'Engine_Volume', 'HasTurbo', 'Cylinders', 'Gear_box_type', 'Drive_wheels', 'Airbags']"

In [61]:
r2a_model_string = "Price ~ C(Manufacturer) + Prod_year + C(Category) + C(Fuel_type) + Engine_Volume + C(HasTurbo) + Cylinders + C(Gear_box_type) + C(Drive_wheels) + Airbags"

In [62]:
r2a_model = smf.ols(r2a_model_string, data = df).fit()

In [63]:
r2a_model.params

Intercept                     -3.034394e+06
C(Manufacturer)[T.AUDI]        3.756034e+03
C(Manufacturer)[T.BMW]         1.310446e+04
C(Manufacturer)[T.BUICK]       1.145679e+03
C(Manufacturer)[T.CADILLAC]    7.885624e+03
                                   ...     
C(Drive_wheels)[T.Rear]        2.747306e+03
Prod_year                      1.512949e+03
Engine_Volume                  1.700510e+03
Cylinders                      1.296979e+03
Airbags                       -8.778789e+02
Length: 70, dtype: float64

In [64]:
press_model_preds = df_results.iloc[28914]

In [65]:
press_model_preds['Predictors']

"['Manufacturer', 'Prod_year', 'Leather_interior', 'HasTurbo', 'Cylinders', 'Gear_box_type', 'Doors', 'Wheel', 'Color', 'Airbags']"

In [66]:
press_model_string = "Price ~ C(Manufacturer) + Prod_year + C(Leather_interior) + C(HasTurbo) + Cylinders + C(Gear_box_type) + C(Doors) + C(Wheel) + C(Color) + Airbags"

In [67]:
press_model = smf.ols(press_model_string, data = df).fit()

In [68]:
press_model.params

Intercept                     -3.048711e+06
C(Manufacturer)[T.AUDI]       -3.016432e+03
C(Manufacturer)[T.BMW]         8.802297e+03
C(Manufacturer)[T.BUICK]      -2.612100e+02
C(Manufacturer)[T.CADILLAC]    2.329693e+03
                                   ...     
C(Color)[T.White]             -7.373351e+02
C(Color)[T.Yellow]             3.922410e+02
Prod_year                      1.517101e+03
Cylinders                      2.140865e+03
Airbags                       -9.927196e+02
Length: 72, dtype: float64

In [69]:
dict_model_params = {'press' : press_model.params, 'r2a' : r2a_model.params, 'bic' : bic_model.params, 'aic' : aic_model.params, 'cv' : cv_model.params}

In [70]:
dict_model_params

{'press': Intercept                     -3.048711e+06
 C(Manufacturer)[T.AUDI]       -3.016432e+03
 C(Manufacturer)[T.BMW]         8.802297e+03
 C(Manufacturer)[T.BUICK]      -2.612100e+02
 C(Manufacturer)[T.CADILLAC]    2.329693e+03
                                    ...     
 C(Color)[T.White]             -7.373351e+02
 C(Color)[T.Yellow]             3.922410e+02
 Prod_year                      1.517101e+03
 Cylinders                      2.140865e+03
 Airbags                       -9.927196e+02
 Length: 72, dtype: float64,
 'r2a': Intercept                     -3.034394e+06
 C(Manufacturer)[T.AUDI]        3.756034e+03
 C(Manufacturer)[T.BMW]         1.310446e+04
 C(Manufacturer)[T.BUICK]       1.145679e+03
 C(Manufacturer)[T.CADILLAC]    7.885624e+03
                                    ...     
 C(Drive_wheels)[T.Rear]        2.747306e+03
 Prod_year                      1.512949e+03
 Engine_Volume                  1.700510e+03
 Cylinders                      1.296979e+03
 Airbags  

In [73]:
for k,v in dict_model_params.items():
    
    v.to_csv(f'{k}.csv', index=True)